In [180]:
import sys
import pandas as pd
import numpy as np
import datetime as dt
# 프로젝트 루트 직접 지정
sys.path.insert(0, r"c:\Users\tjrrj\vscode\doritest")

# 코드

In [ ]:
def gsheet_transformed_df(df):
    df = df.drop(columns=["Unnamed: 0"])
    header_row_idx = 1
    df.columns = df.iloc[header_row_idx]

    df = df.rename(columns=({
        '오픈\n순서' : '오픈순서',
        '변경 \n담당 SV' : '변경담당SV'
    }))
    df = df[2:]
    cond1 = ~df["사업자 번호"].isnull()
    df = df.loc[cond1]

    df['상세주소'] = (
        df['주소'].astype(str)
        # 개행/HTML 특수문자 정리
        .str.replace('\r', ' ', regex=False)
        .str.replace('\n', ' ', regex=False)
        .str.replace('&gt;', '>', regex=False)
        # (이사후 주소) 뒤 텍스트만 추출
        .str.extract(r'\(이사후 주소\)\s*(.+)', expand=False)
        # 없으면 원본에서 (이사전 주소) 라벨 제거한 값으로 대체
        .fillna(
            df['주소'].astype(str)
            .str.replace(r'\(이사전 주소\)\s*', '', regex=True)
            .str.replace('\r', ' ', regex=False)
            .str.replace('\n', ' ', regex=False)
            .str.replace('&gt;', '>', regex=False)
        )
        .str.strip()
    )

    df_col = ['오픈순서', '호점', '매장명', '사업자 번호', '점주명', '변경담당SV',
        '실오픈일', '상세주소', '배달의 민족ID', '배달의 민족PW', '요기요ID', '요기요PW', '쿠팡이츠ID', '쿠팡이츠PW',
        '땡겨요ID', '땡겨요PW', '토더 ID', '토더PW', '네이버 ID', '네이버 pw']

    gsheet_store_list = df[df_col]

    gsheet_store_list["광역"] = gsheet_store_list['상세주소'].str.split(' ').str[0].str[:2]
    gsheet_store_list["시군구"] = gsheet_store_list['상세주소'].str.split(' ').str[1]
    gsheet_store_list["읍면동"] = gsheet_store_list['상세주소'].str.split(' ').str[2]
    col2 = ['오픈순서', '호점', '매장명', '사업자 번호', '점주명', '변경담당SV', '실오픈일', 
        '상세주소', '광역', '시군구', '읍면동',
        '배달의 민족ID', '배달의 민족PW', '요기요ID', '요기요PW', '쿠팡이츠ID', '쿠팡이츠PW', '땡겨요ID',
        '땡겨요PW', '토더 ID', '토더PW', '네이버 ID', '네이버 pw']

    gsheet_store_list = gsheet_store_list[col2]

    # 플랫폼별 컬럼 매핑
    platform_cols = {
        '배달의 민족': ('배달의 민족ID', '배달의 민족PW'),
        '요기요': ('요기요ID', '요기요PW'),
        '쿠팡이츠': ('쿠팡이츠ID', '쿠팡이츠PW'),
        '땡겨요': ('땡겨요ID', '땡겨요PW'),
        '토더': ('토더 ID', '토더PW'),
        '네이버': ('네이버 ID', '네이버 pw')
    }

    # 매장 기본 정보
    keep_cols = ['오픈순서', '호점', '매장명', '사업자 번호', '점주명', "변경담당SV", '실오픈일', '상세주소', '광역', '시군구', '읍면동']

    rows = []
    for _, r in gsheet_store_list.iterrows():
        base = {c: r[c] for c in keep_cols}
        for platform, (id_col, pw_col) in platform_cols.items():
            uid = r.get(id_col)
            pwd = r.get(pw_col)
            if pd.notna(uid) or pd.notna(pwd):  # 둘 중 하나라도 값이 있으면 추가
                rows.append({**base, '플랫폼': platform, '계정ID': uid, '계정PW': pwd})

    # 새 데이터프레임 생성
    accounts_long = pd.DataFrame(rows)

    # 결과 확인
    gsheet_store_list = accounts_long

    gsheet_store_list["collected_at"] = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')


    # 조건에 맞는 행에만 email 값 추가
    gsheet_store_list.loc[gsheet_store_list["매장명"] == "도리당 가락점", "email"] = "a17019@kakao.com"
    gsheet_store_list.loc[gsheet_store_list["매장명"] == "도리당 가락점", "쿠팡_매장명"] = "닭도리탕 전문 도리당 가락점"
    gsheet_store_list.loc[gsheet_store_list["매장명"] == "도리당 가락점", "배민_매장명"] = "[음식배달] 닭도리탕 전문 도리당 가락점"
    gsheet_store_list.rename(columns={"변경담당SV": "담당자"}, inplace=True)
    return gsheet_store_list

In [ ]:
gsheet_transformed_df()